# Visualising BMI Data with `gonum/plot`

In this example we will load the 500 person gender/height/weight index dataset from Kaggle (https://www.kaggle.com/yersever/500-person-gender-height-weight-bodymassindex/version/2) and visualise it using gonum/plot.

In [77]:
import (
    "fmt"
    "github.com/kniren/gota/dataframe"
    "github.com/kniren/gota/series"
	"gonum.org/v1/plot"
	"gonum.org/v1/plot/plotter"
	"gonum.org/v1/plot/plotutil"
	"gonum.org/v1/plot/vg"
    "github.com/gonum/stat/distuv"
    "io/ioutil"
    "bytes"
    "image/color"
    "bufio"
)

In [8]:
const path = "../datasets/bmi/500_Person_Gender_Height_Weight_Index.csv"

In [16]:
b, err := ioutil.ReadFile(path)
if err != nil {
    fmt.Println("Error!", err)
}
df := dataframe.ReadCSV(bytes.NewReader(b))

In [15]:
df

[500x4] DataFrame

    Gender   Height Weight Index
 0: Male     174    96     4    
 1: Male     189    87     2    
 2: Female   185    110    4    
 3: Female   195    104    3    
 4: Male     149    61     3    
 5: Male     189    104    3    
 6: Male     147    92     5    
 7: Male     154    111    5    
 8: Male     174    90     3    
 9: Female   169    103    4    
    ...      ...    ...    ...  
    <string> <int>  <int>  <int>


In [37]:
fmt.Println("Minimum", df.Col("Height").Min())
fmt.Println("Maximum", df.Col("Height").Max())
fmt.Println("Mean", df.Col("Height").Mean())
fmt.Println("Median", df.Col("Height").Quantile(0.5))

Minimum 140
Maximum 199
Mean 169.944
Median 170


11 <nil>

In [39]:
df.Describe()

[7x5] DataFrame

    column   Gender   Height     Weight     Index   
 0: mean     -        169.944000 106.000000 3.748000
 1: stddev   -        16.375261  32.382607  1.355053
 2: min      Female   140.000000 50.000000  0.000000
 3: 25%      -        156.000000 80.000000  3.000000
 4: 50%      -        170.000000 106.000000 4.000000
 5: 75%      -        184.000000 136.000000 5.000000
 6: max      Male     199.000000 160.000000 5.000000
    <string> <string> <float>    <float>    <float> 


In [29]:
// SeriesToPlotValues takes a column of a Dataframe and converts it to a gonum/plot/plotter.Values slice.
// Panics if the column does not exist. 
func SeriesToPlotValues(df dataframe.DataFrame, col string) plotter.Values {
 rows, _ := df.Dims()
 v := make(plotter.Values, rows)
 s := df.Col(col)
 for i := 0; i < rows; i++ {
 v[i] = s.Elem(i).Float()
 }
 return v
}

In [2]:
//  showHistogram plots a histogram of the column with name col in the dataframe df.
func histogramBytes(v plotter.Values, title string) []byte{
// Make a plot and set its title.
	p, err := plot.New()
	if err != nil {
		panic(err)
	}
	p.Title.Text = title
	h, err := plotter.NewHist(v, 10)
	if err != nil {
		panic(err)
	}
    //h.Normalize(1)
	p.Add(h)
    w, err := p.WriterTo(5*vg.Inch, 4*vg.Inch, "jpg")
    if err != nil{
        panic(err)
    }
    var b bytes.Buffer
    writer := bufio.NewWriter(&b)
    w.WriteTo(writer)
    if err := p.Save(5*vg.Inch, 4*vg.Inch, title + ".ong"); err != nil {
		panic(err)
	}
    return b.Bytes()
}

ERROR: repl.go:2:23: undefined "plotter" in plotter.Values <*ast.SelectorExpr>

In [152]:
display.JPEG(histogramBytes(SeriesToPlotValues(df, "Height"), "Height Histogram"))

In [154]:
display.JPEG(histogramBytes(SeriesToPlotValues(df, "Weight"), "Weight Histogram"))

In [156]:
display.JPEG(histogramBytes(SeriesToPlotValues(df, "Index"), "BMI Histogram"))